In [122]:
cd /v/filer4b/v20q001/ebanner/Classes/nlp/Project/irony-classifier/data/conservative/features/bows+punctuation/

/v/filer4b/v20q001/ebanner/Classes/nlp/Project/irony-classifier/data/conservative/features/bows+punctuation


In [1]:
cd /v/filer4b/v20q001/npockrus/NLP/finalProject/venv/src/irony-classifier/data/conservative/features/subreddits

/v/filer4b/v20q001/npockrus/NLP/finalProject/venv/src/irony-classifier/data/conservative/features/subreddits


# Load Comments 

In [2]:
import pickle
import scipy
import numpy as np

with open('features.p', 'r') as f:
    data = pickle.load(f)

In [3]:
# Load comments, labels, and bow vectors
xs = np.array([ comment for comment in sorted(data) ])
ys = np.array([ data[comment]['label'] for comment in sorted(data) ])

subreddits = np.array([data[comment]['subreddits'] for comment in sorted(data)])
subreddits = [ ' '.join(subreddit_list) for subreddit_list in subreddits]
    

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=50000, binary=True, stop_words="english")
#print subreddits
X = vectorizer.fit_transform(subreddits)

In [5]:
print len(xs)
print len(ys)
print vectorizer.get_feature_names()

931
931
[u'1911', u'2007scape', u'2012elections', u'250r', u'321', u'3dprinting', u'3ds', u'40something', u'49ers', u'4chan', u'4x4', u'52weeksofcooking', u'80s', u'90s', u'911dispatchers', u'a_cups', u'abandonedporn', u'ableton', u'abletonlive', u'abuseporn', u'academia', u'acappella', u'accounting', u'acting', u'actuallesbians', u'adamcarolla', u'adelaide', u'adfreetime', u'adhd', u'admincraft', u'adporn', u'ads', u'adultgamers', u'adventuretime', u'advertising', u'advice', u'adviceanimals', u'adviceatheists', u'afol', u'againstmensrights', u'aggies', u'airforce', u'airsoft', u'ak47', u'alaska', u'albany', u'albumaday', u'albumartporn', u'algebra', u'alienblue', u'aliens', u'alisonbrie', u'allthingsprotoss', u'altcountry', u'altnewz', u'ama', u'amateur', u'amateurradio', u'amateurroomporn', u'amazondeals', u'amazonsales', u'amazonunder15', u'amazonunder20', u'americanfootball', u'americanhorrorstory', u'americanpolitics', u'amifreetogo', u'amisexy', u'amiugly', u'amv', u'anal', u'ana

In [8]:
# 5-Fold Test

In [9]:
from sklearn.cross_validation import KFold

kf = KFold(len(xs), n_folds=5, shuffle=True)

# Predictions

In [10]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier

recalls, precisions, f_measures = [], [], []
for train, test in kf:
    # Get training and test data for this round
    Xs_train, Xs_test = X[train], X[test]
    ys_train, ys_test = ys[train], ys[test]

    print Xs_train
    # Train svm
    svm = SGDClassifier(loss="hinge", penalty="l2", class_weight="auto", alpha=.01)
    parameters = { 'alpha': [.001, .01,  .1] }
    clf = GridSearchCV(svm, parameters, scoring='f1')
    clf.fit(Xs_train, ys_train)

    # Make predictions
    predictions = clf.predict(Xs_test)
    
    for test, vector, prediction, actual in zip(xs[test], Xs_test, predictions, ys_test):
        print '========================================================================'
        print
        print 'Comment: {}'.format(test)
        print 'Vectorized as: {}'.format(vector)
        print 'Prediction: {}'.format(prediction)
        print 'Was really: {}'.format(actual)
        print

  (0, 1333)	1
  (0, 1437)	1
  (0, 249)	1
  (0, 345)	1
  (0, 1145)	1
  (0, 292)	1
  (0, 155)	1
  (0, 930)	1
  (0, 223)	1
  (0, 241)	1
  (0, 2083)	1
  (0, 693)	1
  (0, 965)	1
  (0, 813)	1
  (0, 1483)	1
  (0, 793)	1
  (0, 161)	1
  (0, 478)	1
  (1, 1459)	1
  (1, 123)	1
  (1, 1630)	1
  (1, 759)	1
  (1, 1110)	1
  (1, 975)	1
  (1, 1286)	1
  :	:
  (742, 934)	1
  (742, 1234)	1
  (742, 1546)	1
  (742, 734)	1
  (742, 481)	1
  (742, 1110)	1
  (742, 1683)	1
  (742, 1540)	1
  (742, 1274)	1
  (742, 292)	1
  (742, 1973)	1
  (742, 813)	1
  (742, 1483)	1
  (742, 2166)	1
  (742, 2156)	1
  (742, 793)	1
  (742, 161)	1
  (742, 478)	1
  (743, 587)	1
  (743, 1250)	1
  (743, 1833)	1
  (743, 188)	1
  (743, 177)	1
  (743, 813)	1
  (743, 478)	1

Comment: "Alarmism" Really?  How many -isms do you have to make up?
Vectorized as:   (0, 227)	1
  (0, 97)	1
  (0, 109)	1
  (0, 467)	1
  (0, 1911)	1
  (0, 1511)	1
  (0, 1973)	1
  (0, 2070)	1
  (0, 965)	1
  (0, 177)	1
  (0, 813)	1
  (0, 36)	1
  (0, 1483)	1
  (0, 197)	1
  (0

In [13]:
#MOST INFLUENCIAL SUBREDDITS

In [15]:
sgd_clf = clf.best_estimator_
feat_names = vectorizer.get_feature_names()
coefs_with_fns = sorted(zip(sgd_clf.coef_[0], feat_names))
coefs_with_fns[:20]

[(-0.5145930687461574, u'movies'),
 (-0.492576862998996, u'explainlikeimfive'),
 (-0.37205958088953367, u'economics'),
 (-0.34675524051779633, u'mildlyinteresting'),
 (-0.31923498333384415, u'4chan'),
 (-0.29721877758668241, u'liberal'),
 (-0.27598886490191865, u'aww'),
 (-0.27520257183952079, u'minecraft'),
 (-0.25869041752914945, u'beer'),
 (-0.25125637403010792, u'changemyview'),
 (-0.24217826321877828, u'bad_cop_no_donut'),
 (-0.23931901571914557, u'libertarian'),
 (-0.22852535640803781, u'truereddit'),
 (-0.22016205747161666, u'europe'),
 (-0.21973317034667178, u'askscience'),
 (-0.21708836640951248, u'atheism'),
 (-0.21108394666028477, u'worldnews'),
 (-0.19964695666176133, u'breakingbad'),
 (-0.19814585172445504, u'business'),
 (-0.19743103984954211, u'conservative')]

# Performance

## Byron's Results: Precision=0.315 (0.261, 0.380), Recall=0.496 (0.446, 0.548), F-Measure=0.383 (0.330, 0.412)

In [16]:
import sklearn

precisions, recalls, f_measures = [], [], []
for train, test in kf:
    # Train svm
    svm = SGDClassifier(loss="hinge", penalty="l2", class_weight="auto")
    parameters = { 'alpha': [.001, .01,  .1] }
    clf = GridSearchCV(svm, parameters, scoring='f1')
    clf.fit(X[train], ys[train])

    # Make predictions
    predictions = clf.predict(X[test])
    
    # Record statistics
    precision, recall, f_measure, _ = sklearn.metrics.precision_recall_fscore_support(ys[test], predictions, average='binary')
    precisions.append(precision)
    recalls.append(recall)
    f_measures.append(f_measure)
    
print 'Precisions: {}'.format(precisions)
print 'Recalls: {}'.format(recalls)
print 'F-Measures: {}'.format(f_measures)
print
print 'Mean Precision: {}'.format(np.mean(precisions))
print 'Mean Recall: {}'.format(np.mean(recalls))
print 'Mean F-Measure: {}'.format(np.mean(f_measures))

Precisions: [0.15384615384615385, 0.073170731707317069, 0.057692307692307696, 0.13333333333333333, 0.15189873417721519]
Recalls: [0.40000000000000002, 0.17647058823529413, 0.15789473684210525, 0.2857142857142857, 0.5]
F-Measures: [0.22222222222222221, 0.10344827586206896, 0.084507042253521111, 0.18181818181818182, 0.23300970873786409]

Mean Precision: 0.113988252151
Mean Recall: 0.304015922158
Mean F-Measure: 0.165001086179


In [79]:
sgd_clf = clf.best_estimator_
sgd_clf.coef_[0]

array([-0.16719061, -0.16719061,  0.        , ..., -0.66876244,
        0.        ,  0.        ])